# Step1 Import Package & Mount GoogleDrive



In [1]:
!pip install openai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 4.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.3 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
import os
from openai import OpenAI
import base64
import re
from PIL import Image
import requests
import io
import pandas as pd
import time

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Step2 Image Detection Prepare

In [24]:
#@title Function to encode the image and set OpenAI key
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


api_key = "APIKEY"
client = OpenAI(api_key = api_key)

### Get the route of Evaluation Dataset Images

In [34]:
# # For Evaluation
# dir = "/content/drive/Shareddrives/Project_SeminarFall2023 (Prof. Lipen Wang)/Evaluation Dataset Images"

# For Description
dir = "/content/drive/MyDrive/Photo"
file_name = os.listdir(dir)

route = []
for i in file_name:
  i = dir + "/" + i
  route.append(i)
route.sort()
print(route)

['/content/drive/MyDrive/Photo/GBR201600000000001.JPG', '/content/drive/MyDrive/Photo/GBR201600000000005.JPG', '/content/drive/MyDrive/Photo/GBR201600000000006.JPG', '/content/drive/MyDrive/Photo/GBR201600000000007.JPG', '/content/drive/MyDrive/Photo/GBR201600000000009.JPG', '/content/drive/MyDrive/Photo/GBR201600000000010.JPG', '/content/drive/MyDrive/Photo/GBR201600000000011.JPG', '/content/drive/MyDrive/Photo/GBR201600000000013.JPG', '/content/drive/MyDrive/Photo/GBR201600000000016.JPG', '/content/drive/MyDrive/Photo/GBR201600000000017.JPG', '/content/drive/MyDrive/Photo/GBR201600000000018.JPG', '/content/drive/MyDrive/Photo/GBR201600000000019.JPG', '/content/drive/MyDrive/Photo/GBR201600000000024.JPG', '/content/drive/MyDrive/Photo/GBR201600000000025.JPG', '/content/drive/MyDrive/Photo/GBR201600000000026.jpg', '/content/drive/MyDrive/Photo/GBR201600000000033.JPG', '/content/drive/MyDrive/Photo/GBR201600000000037.JPG', '/content/drive/MyDrive/Photo/GBR201600000000041.JPG', '/content

In [14]:
len(route)

175

In [12]:
route = [route[56], route[59], route[76]]
route

['/content/drive/Shareddrives/Project_SeminarFall2023 (Prof. Lipen Wang)/Evaluation Dataset Images/GBR201700000000035.JPG',
 '/content/drive/Shareddrives/Project_SeminarFall2023 (Prof. Lipen Wang)/Evaluation Dataset Images/GBR201700000000039.JPG',
 '/content/drive/Shareddrives/Project_SeminarFall2023 (Prof. Lipen Wang)/Evaluation Dataset Images/GBR201700000000082.JPG']

### Get the Evaluation Submission Excel File

In [35]:
# # For Evaluation
# df = pd.read_excel("/content/drive/MyDrive/Eyes on the Water - Hackathon Model Evaluation submission template.xlsx", index_col = 0)

# For Description
df = pd.read_excel("/content/drive/Shareddrives/Project_SeminarFall2023 (Prof. Lipen Wang)/Development_Dataset/Development_Dataset.xlsx", index_col = 0)
df

,Original_File_Name,File_Path,Image source,Date,Time,Latitude,Longitude,Waterbody,dHash,Variance_of_Laplacian,...,Wildlife_Invertebrate,Wildlife_Birds,Wildlife_Other,Algal_Bloom,Plants,People,Number_Of_People,Sensor,Measurement_Type,Notes
ImageID,,,,,,,,,,,,,,,,,,,,,
GBR201600000000001,DSCF7268,"Site 01 Bradford Beck, Shipley Station",Citizen,2016-11-07,14:31:00,53.834652,-1.771159,Y,12085448196108699648,44.672076,...,N,N,N,N,N,N,0,N,None,NaN
GBR201600000000005,DSCF7277,"Site 02 Bradford Beck, Poplar Road",Citizen,2016-11-15,10:04:00,53.824577,-1.769716,Y,8014066726088689664,713.583815,...,N,Y,N,N,Y,N,0,N,None,NaN
GBR201600000000006,DSCF7299,"Site 09 Middlebrook, Farmer's Boy, Cemetery Road",Citizen,2016-11-15,12:37:00,NaN,NaN,Y,4198524078264309760,1266.500927,...,N,N,N,N,Y,N,0,N,None,NaN
GBR201600000000007,DSCF7346,"Site 01 Bradford Beck, Shipley Station",Citizen,2016-11-24,16:10:00,53.834652,-1.771159,Y,5737074986534179840,379.027533,...,N,N,N,N,Y,N,0,N,None,NaN
GBR201600000000009,DSCF7381,"Site 01 Bradford Beck, Shipley Station",Citizen,2016-11-25,15:44:00,53.824577,-1.769716,Y,5370162433311250432,600.702221,...,N,N,N,N,Y,N,0,N,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
UNKXXXX00000000142,27,Wldlife - Fish,Web search,NaT,NaN,NaN,NaN,Y,11199742788338700288,1365.931256,...,N,N,N,N,N,N,0,N,None,NaN
UNKXXXX00000000143,28,Wldlife - Fish,Web search,NaT,NaN,NaN,NaN,Y,13097937388265199616,530.590775,...,N,N,N,N,N,N,0,N,None,NaN
UNKXXXX00000000144,29,Wldlife - Fish,Web search,NaT,NaN,NaN,NaN,Y,3430964022415410176,506.891938,...,N,N,N,N,N,N,0,N,None,NaN


### Define the Function to Extract the Answer List

In [50]:
def str_ans(result):
  match = re.search(r"\[([^\[\]]+)\]", result)

  if match:
      extracted_response = match.group(1)
      extracted_list = extracted_response.split(",")
      extracted_list = [item.strip() for item in extracted_list]
      print("Extracted list:", extracted_list)

  else:
      print("No matching pattern found.")
  return extracted_list

# Step3 Prompt and Get Answer for the First 7 Questions

In [51]:
df = df.iloc[:, :7]
df

,Outfall,Outfall_Spilling,Outfall_Aeration,Outfall_Screen,Sewage_Litter,Rubbish,Obstruction
ImageID,,,,,,,
GBR201600000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
UNKXXXX00000000125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000127,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
ans = []
k = 0

prompt = ""
# system_content = "The GPT is designed for automated analysis of urban river images, examining seven specific features: 1) Presence of an outfall, which may or may not be a pipe. 2) Flow from the outfall. 3) Aeration caused by the outfall. 4) Screen on the outfall. 5) Presence of sewage 'rags'. 6) General rubbish. 7) Obstruction to river flow. When details are unclear, it infers the most plausible answer using all available data. It provides structured, concise responses, formatted as [Y,N,N,Y,N,Y,Partial], directly corresponding to the analysis of these elements. Specifically, question 7 classified as Full, Partial, Minor, or None. The GPT's approach is professional, accurate, and structured."
system_content = """The GPT is designed for automated analysis of urban river images, examining seven specific features: The first four questions are chain questions, their answer must be identical.
(1) The presence of an outfall. Usually a pipe, but not always.
(2) Is there flow coming out of the outfall?
(3) Is there aeration caused by outfall?
(4) Is there a screen on the outfall?
(5) Presence of sewage 'rags'. For indicating possible sewage pollution.
(6) General rubbish. Cannot count fallen leaves and algae.
(7) An obstruction to river flow. Full = 100% obstruction across channel, Partial >= 25%, Minor <25%, None = 0%.
Specifically, question 7 classified as Full, Partial, Minor, or None. Other questions are classified as Y or N or U, corresponding to Yes, No, and Unknown respectively.
When details are unclear, it infers the most plausible answer using all available data.
It provides structured, concise responses, formatted as [N,N,N,U,N,Y,Partial]. Make sure the response is only in such format without any other description.
And if there exists ambiguities with one specific answer, simply response U instead.
The GPT's approach is professional, accurate, and structured."""

In [53]:
for i in route:

  # Path to your image
  image_path = i
  try:
    # Getting the base64 string
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
      model = "gpt-4-vision-preview",
      messages = [
        {"role": "system", "content": system_content},
          {"role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
              },
            },
          ],
        }
      ],
      max_tokens = 300,
    )
  except:
    print("unsupported image")

  try:
    final_ans = str_ans(response.choices[0].message.content)
    if os.path.basename(i)[:-4] == df.index[k]:
      df.loc[df.index[k]] = final_ans
  except:
    print("No answer")

  print("---", os.path.basename(i), "---")
  print(response.choices[0].message.content)
  print()

  # ans.append(response.choices[0].message.content)
  k += 1
  time.sleep(5)

Extracted list: ['Y', 'Y', 'N', 'N', 'N', 'Y', 'Minor']
--- GBR201600000000002.JPG ---
[Y,Y,N,N,N,Y,Minor]

Extracted list: ['Y', 'Y', 'N', 'N', 'N', 'N', 'None']
--- GBR201600000000003.JPG ---
[Y,Y,N,N,N,N,None]

Extracted list: ['Y', 'Y', 'N', 'N', 'Y', 'Y', 'Partial']
--- GBR201600000000004.JPG ---
[Y,Y,N,N,Y,Y,Partial]

Extracted list: ['Y', 'Y', 'Y', 'N', 'N', 'Y', 'Partial']
--- GBR201600000000008.JPG ---
[Y,Y,Y,N,N,Y,Partial]

Extracted list: ['Y', 'Y', 'Y', 'N', 'N', 'N', 'None']
--- GBR201600000000012.JPG ---
[Y,Y,Y,N,N,N,None]

Extracted list: ['Y', 'Y', 'N', 'N', 'Y', 'Y', 'Partial']
--- GBR201600000000014.JPG ---
[Y,Y,N,N,Y,Y,Partial]

Extracted list: ['N', 'N', 'N', 'N', 'Y', 'Y', 'Partial']
--- GBR201600000000015.JPG ---
[N,N,N,N,Y,Y,Partial]

Extracted list: ['Y', 'Y', 'N', 'N', 'Y', 'Y', 'Partial']
--- GBR201600000000020.jpg ---
[Y,Y,N,N,Y,Y,Partial]

Extracted list: ['Y', 'Y', 'N', 'N', 'Y', 'Y', 'Partial']
--- GBR201600000000022.jpg ---
[Y,Y,N,N,Y,Y,Partial]

Extracte

In [54]:
df

,Outfall,Outfall_Spilling,Outfall_Aeration,Outfall_Screen,Sewage_Litter,Rubbish,Obstruction
ImageID,,,,,,,
GBR201600000000002,Y,Y,N,N,N,Y,Minor
GBR201600000000003,Y,Y,N,N,N,N,None
GBR201600000000004,Y,Y,N,N,Y,Y,Partial
GBR201600000000008,Y,Y,Y,N,N,Y,Partial
GBR201600000000012,Y,Y,Y,N,N,N,None
...,...,...,...,...,...,...,...
UNKXXXX00000000125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000127,Y,Y,Y,N,N,Y,None


In [55]:
df.to_csv("/content/drive/MyDrive/Result.csv")

# Step4 Prompt and Get Answer for the Next 7 Questions

In [59]:
df = df.iloc[:, 7:14]
df

,Modified_Channel,Proportion_Modification,Discolouration,Discolouration_colour,Discolouration_Outfall,Aeration,Aeration_Extent
ImageID,,,,,,,
GBR201600000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...
UNKXXXX00000000125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000127,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [60]:
ans = []
k = 0

prompt = ""
# system_content = "The GPT is tailored for analyzing urban river images, focusing on: 1) if the river channel is man-made or modified, 2) the extent of channel modification (Full, Partial, Minor, None), 3) the presence of water discoloration, 4) the color of the discoloration, 5) if the discoloration is due to an outfall, 6) the presence of visible aeration, and 7) the extent of aeration (Full, Partial, Minor, None). It provides structured, concise responses, formatted as [Y,Partial,N,None,N,Y,Partial]. Specifically, questions 3, 4, and 5 are interrelated: if question 3's answer is 'N' (no discoloration), the responses to questions 4 and 5 will automatically be 'None' and 'N', respectively. The GPT avoids speculation, infers answers from visible information, and communicates formally and accurately."
system_content = """The GPT is tailored for analyzing urban river images, focusing on:
(1) if the river channel is man-made or modified
(2) the extent of channel modification. Full = 100% change to walls and base of channel, Partial >= 25%, Minor <25%, None = 0%.
(3) the presence of water discoloration. Could also indicate high turbidity.
(4) the color of the discoloration
(5) Is the discolouration caused by an outfall
(6) Is there visible aeration in the river reach (e.g. bubbles, turbulence, rapids, weirs…)
(7) the extent of aeration. Full = 100% of visible flow aerated (white water), Partial >= 25%, Minor <25%, None = 0%.
Specifically, question 2 and 7 classified as Full, Partial, Minor, or None. Other questions are classified as Y or N or U, corresponding to Yes, No, and Unknown respectively.
It provides structured, concise responses, formatted as [Y,Partial,N,None,U,Y,Partial]. Make sure the response is only in such format without any other description.
Specifically, questions 3, 4 are interrelated: if question 3's answer is 'N' (no discoloration), the responses to questions 4 will automatically be 'None'.
The GPT avoids speculation, infers answers from visible information, and communicates formally and accurately."""

In [61]:
for i in route:
  # Path to your image
  image_path = i
  try:
    # Getting the base64 string
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
      model = "gpt-4-vision-preview",
      messages = [
        {"role": "system", "content": system_content},
          {"role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
              },
            },
          ],
        }
      ],
      max_tokens = 300,
    )
  except:
    print("unsupported image")

  try:
    final_ans = str_ans(response.choices[0].message.content)
    if os.path.basename(i)[:-4] == df.index[k]:
      df.loc[df.index[k]] = final_ans
  except:
    print("No answer")

  print("---", os.path.basename(i), "---")
  print(response.choices[0].message.content)
  print()

  # ans.append(response.choices[0].message.content)
  k += 1
  time.sleep(5)

Extracted list: ['Y', 'Partial', 'N', 'None', 'N', 'N', 'None']
--- GBR201600000000002.JPG ---
[Y,Partial,N,None,N,N,None]

Extracted list: ['Y', 'Partial', 'Y', 'Brown', 'Y', 'N', 'None']
--- GBR201600000000003.JPG ---
[Y,Partial,Y,Brown,Y,N,None]

Extracted list: ['Y', 'Partial', 'N', 'None', 'N', 'Minor', 'Partial']
--- GBR201600000000004.JPG ---
[Y,Partial,N,None,N,Minor,Partial]

Extracted list: ['Y', 'Full', 'N', 'None', 'N', 'N', 'None']
--- GBR201600000000008.JPG ---
[Y,Full,N,None,N,N,None]

Extracted list: ['Y', 'Full', 'Y', 'Brown', 'Y', 'N', 'None']
--- GBR201600000000012.JPG ---
[Y,Full,Y,Brown,Y,N,None]

Extracted list: ['Y', 'Partial', 'N', 'None', 'N', 'N', 'None']
--- GBR201600000000014.JPG ---
[Y,Partial,N,None,N,N,None]

Extracted list: ['Y', 'Partial', 'N', 'None', 'N', 'N', 'None']
--- GBR201600000000015.JPG ---
[Y,Partial,N,None,N,N,None]

Extracted list: ['Y', 'Full', 'N', 'None', 'N', 'N', 'None']
--- GBR201600000000020.jpg ---
[Y,Full,N,None,N,N,None]

Extracte

In [62]:
df

,Modified_Channel,Proportion_Modification,Discolouration,Discolouration_colour,Discolouration_Outfall,Aeration,Aeration_Extent
ImageID,,,,,,,
GBR201600000000002,Y,Partial,N,None,N,N,None
GBR201600000000003,Y,Partial,Y,Brown,Y,N,None
GBR201600000000004,Y,Partial,N,None,N,Minor,Partial
GBR201600000000008,Y,Full,N,None,N,N,None
GBR201600000000012,Y,Full,Y,Brown,Y,N,None
...,...,...,...,...,...,...,...
UNKXXXX00000000125,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000127,Y,Full,Y,Brown,Y,Y,Full


In [63]:
df.to_csv("/content/drive/MyDrive/Result_2.csv")

# Step5 Prompt and Get Answer for the Last 8 Questions

In [68]:
df = df.iloc[:, 14:]
df

,Sensor,Wildlife,Wildlife_Fish,Wildlife_Algal,Wildlife_Invertebrate,Wildlife_Birds,Wildlife_Other,Plants
ImageID,,,,,,,,
GBR201600000000002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000003,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000004,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
GBR201600000000012,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...
UNKXXXX00000000125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
UNKXXXX00000000127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [69]:
def str_ans(result):
  match = re.search(r"\[([^\[\]]+)\]", result)

  if match:
      extracted_response = match.group(1)
      extracted_list = extracted_response.split(",")
      extracted_list = [item.strip() for item in extracted_list]
      print("Extracted list:", extracted_list)

      element_to_move = extracted_list.pop(6)
      extracted_list.insert(1, element_to_move)
      print("Modified Extracted list:", extracted_list)

  else:
      print("No matching pattern found.")

  return extracted_list

In [70]:
ans = []
k = 0

prompt = ""
# system_content = "The GPT is tailored for analyzing urban river images, focusing on: 1) if the river channel is man-made or modified, 2) the extent of channel modification (Full, Partial, Minor, None), 3) the presence of water discoloration, 4) the color of the discoloration, 5) if the discoloration is due to an outfall, 6) the presence of visible aeration, and 7) the extent of aeration (Full, Partial, Minor, None). It provides structured, concise responses, formatted as [Y,Partial,N,None,N,Y,Partial]. Specifically, questions 3, 4, and 5 are interrelated: if question 3's answer is 'N' (no discoloration), the responses to questions 4 and 5 will automatically be 'None' and 'N', respectively. The GPT avoids speculation, infers answers from visible information, and communicates formally and accurately."
system_content = """The GPT is designed for automated analysis of urban river images, examining the presence of 8 specific features:
(1) Presence of flow gauges, levels, or quality sensors. Any sort of flow or water quality measurement device detected in the photo
(2) Are fish species present
(3) Are algal blooms present
(4) Are invertebrate species present
(5) Are birds present
(6) Are other forms of wildlife present
(7) Is there wildlife present
(8) Are plant species present
When details are unclear, it infers the most plausible answer using all available data. It provides structured, concise responses, formatted as [N,Y,N,N,Y,N,N,N], in the order of which 'Y' stands for ‘Yes’ and 'N' stands for ‘No’. Make sure the response is only in such format without any other description.
Question (2 - 7) are interrelated about the presence of wildlife. If any answer (2 to 6) is 'Y', Answer of (7) must be 'Y'.
The GPT's approach is professional, accurate, and structured."""

In [71]:
for i in route:
  # Path to your image
  image_path = i
  try:
    # Getting the base64 string
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
      model = "gpt-4-vision-preview",
      messages = [
        {"role": "system", "content": system_content},
          {"role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
              },
            },
          ],
        }
      ],
      max_tokens = 300,
    )
  except:
    print("unsupported image")

  try:
    final_ans = str_ans(response.choices[0].message.content)
    if os.path.basename(i)[:-4] == df.index[k]:
      df.loc[df.index[k]] = final_ans
  except:
    print("No answer")

  print("---", os.path.basename(i), "---")
  print(response.choices[0].message.content)
  print()

  # ans.append(response.choices[0].message.content)
  k += 1
  time.sleep(5)

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'Y', 'N']
Modified Extracted list: ['N', 'Y', 'N', 'N', 'N', 'N', 'N', 'N']
--- GBR201600000000002.JPG ---
[N,N,N,N,N,N,Y,N]

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'Y', 'Y']
Modified Extracted list: ['N', 'Y', 'N', 'N', 'N', 'N', 'N', 'Y']
--- GBR201600000000003.JPG ---
[N,N,N,N,N,N,Y,Y]

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y']
Modified Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y']
--- GBR201600000000004.JPG ---
[N,N,N,N,N,N,N,Y]

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
Modified Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
--- GBR201600000000008.JPG ---
[N,N,N,N,N,N,N,N]

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
Modified Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'N']
--- GBR201600000000012.JPG ---
[N,N,N,N,N,N,N,N]

Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y']
Modified Extracted list: ['N', 'N', 'N', 'N', 'N', 'N', 'N', 'Y']
--- GBR20160

In [72]:
df

,Sensor,Wildlife,Wildlife_Fish,Wildlife_Algal,Wildlife_Invertebrate,Wildlife_Birds,Wildlife_Other,Plants
ImageID,,,,,,,,
GBR201600000000002,N,Y,N,N,N,N,N,N
GBR201600000000003,N,Y,N,N,N,N,N,Y
GBR201600000000004,N,N,N,N,N,N,N,Y
GBR201600000000008,N,N,N,N,N,N,N,N
GBR201600000000012,N,N,N,N,N,N,N,N
...,...,...,...,...,...,...,...,...
UNKXXXX00000000125,N,N,Y,N,N,N,Y,Y
UNKXXXX00000000126,N,Y,Y,N,N,N,N,N
UNKXXXX00000000127,N,Y,N,N,N,N,N,N


In [73]:
df.to_csv("/content/drive/MyDrive/Result_3.csv")

# ----------------------------------------------------------------------------------------------------------------------

# Photo Description for Development_Dataset

In [36]:
df = df.iloc[:, :1]
df.insert(0, "Description", [None]*175)
df = df.iloc[:, :1]
df

,Description
ImageID,
GBR201600000000001,None
GBR201600000000005,None
GBR201600000000006,None
GBR201600000000007,None
GBR201600000000009,None
...,...
UNKXXXX00000000142,None
UNKXXXX00000000143,None
UNKXXXX00000000144,None


In [37]:
ans = []
k = 0

prompt = "No more than 100 words"
system_content = """The GPT specializes in analyzing urban river photographs, focusing on environmental indicators. It identifies and describes elements like outfall, sewage litter, discoloration, obstructions, channel modifications, aeration, sensors, and wildlife in no more than 100 words. The GPT provides objective, factual observations from the photograph without speculative interpretations. The responses are concise, informative, and suitable for both environmental professionals and enthusiasts. The GPT will ask for clarifications if the photo's details are ambiguous, ensuring accurate and succinct analyses. The tone remains informative, catering to a varied audience interested in environmental observations of urban rivers."""

In [38]:
for i in route:
  # Path to your image
  image_path = i
  try:
    # Getting the base64 string
    base64_image = encode_image(image_path)
    response = client.chat.completions.create(
      model = "gpt-4-vision-preview",
      messages = [
        {"role": "system", "content": system_content},
          {"role": "user",
          "content": [
            {"type": "text", "text": prompt},
            {
              "type": "image_url",
              "image_url": {
                "url": f"data:image/jpeg;base64,{base64_image}",
                "detail": "high"
              },
            },
          ],
        }
      ],
      max_tokens = 200,
    )
  except:
    print("unsupported image")

  try:
    # final_ans = str_ans(response.choices[0].message.content)
    if os.path.basename(i)[:-4] == df.index[k]:
      df.loc[df.index[k]] = response.choices[0].message.content
  except:
    print("No answer")

  print("---", os.path.basename(i), "---")
  print(response.choices[0].message.content)
  print()

  # ans.append(response.choices[0].message.content)
  k += 1
  time.sleep(5)

--- GBR201600000000001.JPG ---
This urban river shows an outfall with bars likely designed to prevent large debris entry. Graffiti is present on walls, indicating human interaction with the site. There is litter accumulation near the outfall and on the river banks, pointing to pollution issues. The water appears to have discoloration and the presence of foam suggests possible contamination. Channel modifications are evident with concrete structures, which could alter natural flow. The area seems poorly maintained, raising concerns for potential ecological impacts. No wildlife can be observed in this photograph.

--- GBR201600000000005.JPG ---
This urban river section shows riparian vegetation and some channel modifications such as brick lining on the right bank. There's direct surface run-off visible on the left, with possible outfall entry points discernible. Waterfowl are present, indicating some level of ecological activity. No significant visible sewage litter or discoloration. Can

In [39]:
df

,Description
ImageID,
GBR201600000000001,This urban river shows an outfall with bars li...
GBR201600000000005,This urban river section shows riparian vegeta...
GBR201600000000006,This photo presents a river flowing through an...
GBR201600000000007,The river shows moderate flow and is confined ...
GBR201600000000009,The photograph captures a deteriorating urban ...
...,...
UNKXXXX00000000142,The image displays a fish leaping in turbulent...
UNKXXXX00000000143,"The image displays a group of fish, likely spa..."
UNKXXXX00000000144,The photograph shows a river with dead fish in...


In [40]:
df.to_csv("/content/drive/MyDrive/Description.csv")

# Automated Photo Description for App

In [1]:
!pip install openai pillow

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 221.5/221.5 kB 2.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.0/75.0 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.9/76.9 kB 10.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 6.1 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
llmx 0.0.15a0 requires cohere, which is not installed.
llmx 0.0.15a0 requires tiktoken, which is not installed.


In [2]:
from openai import OpenAI
import base64

In [3]:
#@title Function to encode the image and set OpenAI key
def encode_image(image_path):
  with open(image_path, "rb") as image_file:
    return base64.b64encode(image_file.read()).decode('utf-8')


api_key = "APIKEY"
client = OpenAI(api_key = api_key)

In [4]:
prompt = "No more than 100 words"
system_content = """The GPT specializes in analyzing urban river photographs, focusing on environmental indicators. It identifies and describes elements like outfall, sewage litter, discoloration, obstructions, channel modifications, aeration, sensors, and wildlife in no more than 100 words. The GPT provides objective, factual observations from the photograph without speculative interpretations. The responses are concise, informative, and suitable for both environmental professionals and enthusiasts. The GPT will ask for clarifications if the photo's details are ambiguous, ensuring accurate and succinct analyses. The tone remains informative, catering to a varied audience interested in environmental observations of urban rivers."""

In [7]:
# !!!Please insert the image path here
image_path = '/content/drive/MyDrive/Photo/GBR201600000000001.JPG'

# Getting the base64 string
base64_image = encode_image(image_path)

response = client.chat.completions.create(
  model="gpt-4-vision-preview",
  messages=[
    {"role": "system", "content": system_content},
      {"role": "user",
      "content": [
        {"type": "text", "text": prompt},
        {
          "type": "image_url",
          "image_url": {
            "url": f"data:image/jpeg;base64,{base64_image}",
            "detail": "high"
          },
        },
      ],
    }
  ],
  max_tokens = 200,
)

# Here is the result
Description = response.choices[0].message.content
print(Description)

This urban river photograph shows a channel influenced by human activity, featuring graffiti, litter, and debris accumulation. Notable is an outfall structure possibly for stormwater, evident by the barred tunnel and dark water. Discoloration is observed in water pools, indicating potential pollution. The channel's concrete modification affects natural flow, possibly impacting local ecosystems. While visible litter includes cardboard, which poses a contamination risk. There is no clear presence of wildlife or aeration devices. Further evaluation of sensor installations or water quality would be beneficial for a comprehensive environmental assessment.
